In [20]:
import tensorflow as tf
from random import shuffle
import numpy as np

In [22]:
data = np.load('data.npy', allow_pickle=True)
shuffle(data)

# 80% - 20% split for train and test data
train = data[:39936]
test = data[39936:]

train_X = []
train_y = []
for x in train:
    train_X.append(x[0])
    train_y.append(x[1])

test_X = []
test_y = []
for x in test:
    test_X.append(x[0])
    test_y.append(x[1])

train_X = np.array(train_X)
train_y = np.array(train_y)

test_X = np.array(test_X)
test_y = np.array(test_y)

In [24]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

epochs=25
batch_size=32

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(train_X.shape[1], train_X.shape[2], train_X.shape[3])),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(78, activation='softmax')
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 178, 178, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 89, 89, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 87, 87, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 18, 18, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 10368)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10368)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     5,308,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 78)             │        40,014 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,607,950 (21.39 MB)

 Trainable params: 5,607,950 (21.39 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
cp = tf.keras.callbacks.ModelCheckpoint(filepath="25epochs_conv_newdataset.keras",
                               save_best_only=True,
                               verbose=0)

model.compile(loss = 'sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, 
                    validation_data=(test_X, test_y), callbacks=[cp]).history

Epoch 1/25
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 918s 730ms/step - accuracy: 0.5514 - loss: 1.9644 - val_accuracy: 0.8545 - val_loss: 6.5893
Epoch 2/25
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 896s 718ms/step - accuracy: 0.9953 - loss: 0.0299 - val_accuracy: 0.8680 - val_loss: 4.8777
Epoch 3/25
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 882s 707ms/step - accuracy: 0.9987 - loss: 0.0109 - val_accuracy: 0.8879 - val_loss: 4.4869
Epoch 4/25
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 879s 704ms/step - accuracy: 0.9991 - loss: 0.0052 - val_accuracy: 0.8872 - val_loss: 5.8776
Epoch 5/25
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 878s 703ms/step - accuracy: 0.9991 - loss: 0.0055 - val_accuracy: 0.8884 - val_loss: 6.2697
Epoch 6/25
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 877s 702ms/step - accuracy: 0.9993 - loss: 0.0066 - val_accuracy: 0.8856 - val_loss: 10.7408
Epoch 7/25
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 934s 749ms/step - accuracy: 0.9986 - loss: 0.0130 - val_accuracy: 0.8848 - val_loss: 7.9278
Epoch 8/25
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 883s 708ms/step - a